In [1]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime

In [2]:
import sys
import os
root_path = os.path.abspath(os.path.join(os.getcwd(), "../.."))
if root_path not in sys.path:
    sys.path.append(root_path)
from fileDir import getDataDir, getModelDir, getPredDir
from modules.v1.xgboost import trainTestXgboost

In [3]:
VERSION = 1
TRAIN_PATH = getDataDir("train")
TEST_PATH = getDataDir("test")
CLEANED_TRAIN_PATH = getDataDir("train", VERSION)
CLEANED_TEST_PATH = getDataDir("test", VERSION)

## Clean Datasets

In [4]:
def cleanDataset(df, cleanPath):
    # Drop unnecessary columns (keep ID)
    drop_cols = ['pms_i_ymd','Area','Province','Shop Name','date_of_birth_week',
                 'c_postal_code','c_date_of_salary_payment','media',
                 'place_for_sending_information','r_generalcode4']
    df = df.drop(columns=drop_cols, errors='ignore')

    # Combine year/month features safely
    if {'living_period_month','living_period_year'}.issubset(df.columns):
        df['living_period_month'] = df['living_period_month'].fillna(0) + df['living_period_year'].fillna(0) * 12
        df.drop(columns=['living_period_year'], inplace=True)

    if {'c_number_of_working_month','c_number_of_working_year'}.issubset(df.columns):
        df['c_number_of_working_month'] = df['c_number_of_working_month'].fillna(0) + df['c_number_of_working_year'].fillna(0) * 12
        df.drop(columns=['c_number_of_working_year'], inplace=True)

    # Convert birth date to age
    if 'date_of_birth' in df.columns:
        df['date_of_birth'] = pd.to_datetime(df['date_of_birth'], errors='coerce')
        df['age'] = 2025 - df['date_of_birth'].dt.year
        df.drop(columns=['date_of_birth'], inplace=True)

    # Ensure numeric columns are numeric
    num_cols = ['number_of_children','number_of_resident','living_period_month','c_number_of_employee',
                'c_monthly_salary','c_number_of_working_month','r_expected_credit_limit',
                'r_allloan_case','r_allloan_amount','r_additional_income','r_spouse_income','age']
    for c in num_cols:
        if c in df.columns:
            df[c] = pd.to_numeric(df[c], errors='coerce')
    num_cols = [c for c in num_cols if c in df.columns]
    df[num_cols] = df[num_cols].fillna(df[num_cols].median())

    # Fill categorical columns
    cat_cols = ['gender','marital_status','postal_code','tel_category','type_of_residence',
                'c_business_type','c_position','c_occupation','c_employment_status',
                'c_salary_payment_methods','r_propose','r_generalcode1','r_generalcode2',
                'r_generalcode3','r_generalcode5','apply']
    for c in cat_cols:
        if c in df.columns:
            df[c] = df[c].fillna('Unknown')

    print(df.head(3))
    df.to_csv(cleanPath, index=False)
    return df

In [5]:
train_df = pd.read_csv(TRAIN_PATH)
test_df = pd.read_csv(TEST_PATH)
ids = test_df["ID"].copy()

train_df = cleanDataset(train_df, CLEANED_TRAIN_PATH)
test_df = cleanDataset(test_df, CLEANED_TEST_PATH)

             ID gender  marital_status  number_of_children  postal_code  \
0  2.024120e+11     F2               1                   2        10400   
1  2.024120e+11      M               1                   0        10500   
2  2.024120e+11     F2               1                   0        10170   

   tel_category  number_of_resident  living_period_month  type_of_residence  \
0             3                   2                   60                  6   
1             3                   3                   48                  6   
2             3                   6                   62                  3   

   c_business_type  ...  r_allloan_amount  r_additional_income  \
0                7  ...                 0                  0.0   
1                7  ...                 0               5000.0   
2                7  ...             10000                  0.0   

   r_spouse_income  r_generalcode1  r_generalcode2  r_generalcode3  \
0              0.0         Unknown         Unkn

## Testing

In [6]:
trainTestXgboost(VERSION, train_df, test_df, ids)

Tuning hyperparameters...
Fitting 3 folds for each of 25 candidates, totalling 75 fits


c:\Users\Toa\anaconda3\Lib\site-packages\xgboost\training.py:199: UserWarning: [17:09:30] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:790: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


Best Parameters Found: {'subsample': 0.9, 'n_estimators': 400, 'min_child_weight': 0.4, 'max_depth': 10, 'learning_rate': 0.02, 'gamma': 0.3, 'colsample_bytree': 1.0}

Optimal Threshold: 0.137

Confusion Matrix:
[[3540 2123]
 [ 374  468]]

Classification Report:
              precision    recall  f1-score   support

           0       0.90      0.63      0.74      5663
           1       0.18      0.56      0.27       842

    accuracy                           0.62      6505
   macro avg       0.54      0.59      0.51      6505
weighted avg       0.81      0.62      0.68      6505

ROC-AUC Score: 0.6198446976099807
